In [4]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point

# --- Step 1: Load the health facilities ---
df = pd.read_csv("/Users/ekowaddai/Documents/GitHub/Personal/Geojson/UKRANIE_FACILITIES_JUN 9.csv")  # must have 'latitude' and 'longitude' columns

# Convert to GeoDataFrame
gdf_points = gpd.GeoDataFrame(
    df,
    geometry=gpd.points_from_xy(df.longitude, df.latitude),
    crs="EPSG:4326"
)

# --- Step 2: Load admin boundary GeoJSONs ---
gdf_adm1 = gpd.read_file("/Users/ekowaddai/Documents/GitHub/Personal/Geojson/geoBoundaries-UKR-ADM1.geojson")  # Oblasts
gdf_adm2 = gpd.read_file("/Users/ekowaddai/Documents/GitHub/Personal/Geojson/geoBoundaries-UKR-ADM2.geojson")  # Raions

# Ensure same CRS
gdf_adm1 = gdf_adm1.to_crs("EPSG:4326")
gdf_adm2 = gdf_adm2.to_crs("EPSG:4326")

# Rename the name columns for clarity
gdf_adm1 = gdf_adm1.rename(columns={"shapeName": "ADM1"})
gdf_adm2 = gdf_adm2.rename(columns={"shapeName": "ADM2"})

# --- Step 3: Spatial join ADM1 ---
gdf_with_adm1 = gpd.sjoin(gdf_points, gdf_adm1[['ADM1', 'geometry']], how="left", predicate="within")

# 🔧 Drop 'index_right' to avoid conflict in second join
gdf_with_adm1 = gdf_with_adm1.drop(columns=['index_right'], errors='ignore')

# --- Step 4: Spatial join ADM2 ---
gdf_with_adm2 = gpd.sjoin(gdf_with_adm1, gdf_adm2[['ADM2', 'geometry']], how="left", predicate="within")

# --- Step 5: Export final result ---
result = gdf_with_adm2[['HEALTH_FACILITY_NAME', 'latitude', 'longitude', 'ADM1', 'ADM2']]
result.to_csv("facilities_with_oblasts_and_raions.csv", index=False)
